<a href="https://colab.research.google.com/github/tianiachan/IdentifyingAirBnBsinSantaClara/blob/main/Data670_Identifying_AirBnBs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#upload necessary csv to google drive 
from google.colab import files
uploaded = files.upload()

Saving calendar.csv to calendar.csv
Saving listings.csv to listings.csv
Saving reviews.csv to reviews.csv


In [61]:
#import as dataframe 
import pandas as pd
import io
listings_df = pd.read_csv('/content/listings.csv')
calendar_df = pd.read_csv('/content/calendar.csv')
reviews_df = pd.read_csv('/content/reviews.csv')

# Step by Step EDA in Python https://towardsdatascience.com/exploratory-data-analysis-in-python-a-step-by-step-process-d0dfa6bf94ee

In [62]:
#drop columns not needed for listings
listings_df.columns
listings_df=listings_df.drop(columns=['host_id','host_name','neighbourhood_group','price','minimum_nights','number_of_reviews','last_review','reviews_per_month','availability_365','number_of_reviews_ltm'])

In [63]:
#rename id column to listing id to join later
listings_df.rename(columns={'id':'listing_id'}, inplace=True)

In [64]:
listings_df.head()

,listing_id,name,neighbourhood,latitude,longitude,room_type,calculated_host_listings_count,license
0,2861174.0,Casablanca Master Suite,Palo Alto,37.41710,-122.12669,Private room,1,NaN
1,1364787.0,Iris House Cozy room in dwntn Palo Alto & Stan...,Palo Alto,37.44973,-122.15081,Private room,5,NaN
2,2880487.0,**Premium Executive Condo. with Maid Svcs**,Los Altos,37.37318,-122.10736,Entire home/apt,1,NaN
3,1366049.0,Beautiful Single Family Home,Campbell,37.26901,-121.96325,Private room,1,NaN
4,4952.0,Butterfly Inn near Stanford with weekly cleaning,Palo Alto,37.43932,-122.15741,Private room,5,NaN


In [65]:
#same for reviews
reviews_df.columns

Index(['listing_id', 'id', 'date', 'reviewer_id', 'reviewer_name', 'comments'], dtype='object')

In [66]:
#drop the columns
reviews_df=reviews_df.drop(columns=['reviewer_id'])

In [67]:
reviews_df.head()

,listing_id,id,date,reviewer_name,comments
0,4952,6560,2009-08-02,Diane,Staying with Mei Lin and Ross at the Butterfly...
1,1353178,7471795,2013-09-21,Jesus,I will definitely stay at HackerHome when I go...
2,1353178,10120145,2014-02-01,Jeanne,Rob and Vic = family.\r<br/>Class acts.
3,1353178,11096627,2014-03-20,Debora,"Very nice place, nice people, nice location! I..."
4,1353178,11875945,2014-04-17,Pascal,I had a really good time in the HackerHome. Ev...


In [68]:
#rename column id so it says review id
reviews_df.rename(columns={'id':'review_id'}, inplace=True)

In [69]:
#check that changes happened
reviews_df.head()

,listing_id,review_id,date,reviewer_name,comments
0,4952,6560,2009-08-02,Diane,Staying with Mei Lin and Ross at the Butterfly...
1,1353178,7471795,2013-09-21,Jesus,I will definitely stay at HackerHome when I go...
2,1353178,10120145,2014-02-01,Jeanne,Rob and Vic = family.\r<br/>Class acts.
3,1353178,11096627,2014-03-20,Debora,"Very nice place, nice people, nice location! I..."
4,1353178,11875945,2014-04-17,Pascal,I had a really good time in the HackerHome. Ev...


In [70]:
#do the same for calendar
calendar_df.columns

Index(['listing_id', 'date', 'available', 'price', 'adjusted_price',
       'minimum_nights', 'maximum_nights'],
      dtype='object')

In [71]:
calendar_df=calendar_df.drop(columns=['price','adjusted_price','minimum_nights','maximum_nights'])

In [72]:
#exploratory statistics on each dataset
listings_df.describe()

,listing_id,latitude,longitude,calculated_host_listings_count
count,6.162000e+03,6162.000000,6162.000000,6162.000000
mean,2.086868e+17,37.346771,-121.961575,54.354755
std,3.186442e+17,0.066625,0.111997,130.668417
min,4.952000e+03,36.965600,-122.196520,1.000000
25%,2.599622e+07,37.317750,-122.039510,1.000000
50%,4.625849e+07,37.348895,-121.951896,3.000000
75%,6.028115e+17,37.394208,-121.885593,13.000000
max,7.901690e+17,37.462000,-121.380120,461.000000


In [73]:
#paired with datatype and identification of null values
listings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6162 entries, 0 to 6161
Data columns (total 8 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   listing_id                      6162 non-null   float64
 1   name                            6161 non-null   object 
 2   neighbourhood                   6162 non-null   object 
 3   latitude                        6162 non-null   float64
 4   longitude                       6162 non-null   float64
 5   room_type                       6162 non-null   object 
 6   calculated_host_listings_count  6162 non-null   int64  
 7   license                         258 non-null    object 
dtypes: float64(3), int64(1), object(4)
memory usage: 385.2+ KB


In [74]:
#finally check for any duplicates?
listings_df.duplicated().sum()

14

In [75]:
reviews_df.describe() #no staistical value since the rest are boolean or string

,listing_id,review_id
count,1.966170e+05,1.966170e+05
mean,2.610696e+16,2.208022e+17
std,1.263996e+17,2.991275e+17
min,4.952000e+03,6.560000e+03
25%,1.101710e+07,3.223056e+08
50%,2.215150e+07,6.068444e+08
75%,3.818878e+07,5.363594e+17
max,7.803523e+17,7.907592e+17


In [76]:
reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196617 entries, 0 to 196616
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   listing_id     196617 non-null  int64 
 1   review_id      196617 non-null  int64 
 2   date           196617 non-null  object
 3   reviewer_name  196616 non-null  object
 4   comments       196564 non-null  object
dtypes: int64(2), object(3)
memory usage: 7.5+ MB


In [77]:
reviews_df.duplicated().sum()

0

In [78]:
calendar_df.describe()

,listing_id
count,2.249130e+06
mean,2.086868e+17
std,3.186184e+17
min,4.952000e+03
25%,2.598223e+07
50%,4.625849e+07
75%,6.028160e+17
max,7.901690e+17


In [79]:
calendar_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2249130 entries, 0 to 2249129
Data columns (total 3 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   listing_id  int64 
 1   date        object
 2   available   object
dtypes: int64(1), object(2)
memory usage: 51.5+ MB


In [80]:
calendar_df.duplicated().sum()

0

**Beginning of section for own notes - DO NOT USE**

In [81]:
#concentrate on the text cleaning now
#takenfrom https://www.analyticsvidhya.com/blog/2022/01/text-cleaning-methods-in-nlp/
#lowercaseing the data, need as type str so that it processes as string even though datatype is object
reviews_df['comments'] = reviews_df['comments'].astype('str').apply(str.lower)

In [82]:
#test dataframe for testing methods before executing on reviews_df
#reviews_test = reviews_df.copy()

In [83]:
#preview the changes
#reviews_test.head(20)

In [84]:
#remove the <br/> from the reviews
reviews_df['comments'] = reviews_df['comments'].str.replace('<br/>','')

In [85]:
#get rid of the \r
reviews_df['comments'] = reviews_df['comments'].str.replace('\r','')

In [86]:
#remove punctuation
import string
#method
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text
#removal from column
reviews_df['comments'] = reviews_df['comments'].astype('str').apply(remove_punctuations)

In [87]:
#remove numbers
reviews_df['comments'] = reviews_df['comments'].str.replace('\d+', '')

<ipython-input-87-ff7a0831e770>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  reviews_df['comments'] = reviews_df['comments'].str.replace('\d+', '')


In [88]:
#remove extra spaces
reviews_df['comments'] = reviews_df['comments'].str.strip()

In [89]:
#remove emojis
import re
#method and library needed to remove emojis
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"U0001F600-U0001F64F"  # emoticons
                           u"U0001F300-U0001F5FF"  # symbols & pictographs
                           u"U0001F680-U0001F6FF"  # transport & map symbols
                           u"U0001F1E0-U0001F1FF"  # flags (iOS)
                           u"U00002702-U000027B0"
                           u"U000024C2-U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

In [90]:
# actual removal
reviews_df['comments'] = reviews_df['comments'].astype('str').apply(remove_emoji)

In [91]:
#run this each new run then comment it out
!pip install contractions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [92]:
#remove contractions
#pip install library and the method to remove contractions
import contractions
#empty list for expanded text
def remove_contractions(text):
  return contractions.fix(text)

In [93]:
#actually remove
reviews_df['comments'] = reviews_df['comments'].astype('str').apply(remove_contractions)

In [94]:
#remove stopwords, tokenize and create a wordcloud with the words that are most frequent.. later
import nltk
nltk.download('all')
import wordcloud
from nltk import sent_tokenize as st
from nltk import word_tokenize as wt
from nltk.probability import FreqDist as fd
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS

stop_words = nltk.corpus.stopwords.words('english')
print(stop_words)


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    |   Package bcp47 is already up-to-dat

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data]    |   Package wordnet_ic is already up-to-date!
[nltk_data]    | Downloading package words to /root/nltk_data...
[nltk_data]    |   Package words is already up-to-date!
[nltk_data]    | Downloading package ycoe to /root/nltk_data...
[nltk_data]    |   Package ycoe is already up-to-date!
[nltk_data]    | 
[nltk_data]  Done downloading collection all


In [95]:
#remove stop words and words that are only one character long
#reviews_df['comments'] = reviews_df['comments'].astype('str').apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

reviews_df['comments'] = reviews_df['comments'].astype('str').apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words) and len(word) != 1]))

In [96]:
#remove where only 1 character long, most likely typo or nonsensical HELP MEEEEEEEEEEEEEEEEEEEEEEEEEEEEEE
#reviews_df['comments'] = reviews_df['comments'].astype('str').apply(lambda x: ' '.join([word for word in x.split() if len(word) != 1]))

In [97]:
##print to csv for use in tableau and SAS Enterprise Miner
reviews_df.to_csv("reviewMod.csv")

In [98]:
listings_df.to_csv("listingsMod.csv")

In [99]:
calendar_df.to_csv("calendarMod.csv")

**jk stop here**

In [ ]:
#get frequency of each word?
#first turn into string
commentStr = ' '.join(reviews_df['comments'])


In [ ]:
# https://www.guru99.com/tokenize-words-sentences-nltk.html -- tokenize sentence if wat to see average word per sentence
#how to https://www.geeksforgeeks.org/introduction-to-nltk-tokenization-stemming-lemmatization-pos-tagging/
#then tokenize for sentence (in case) and words
sent_comment = st(commentStr)

In [ ]:
type(sent_comment)

list

In [ ]:
#for words
word_comment = wt(commentStr)

In [ ]:
type(word_comment)

list

In [ ]:
#test the contents
for i in range(5):
  print(word_comment[i])

staying
mei
lin
ross
butterfly


In [ ]:
#convert to string for lemmatization
# convert from array of strings to a giant string
wordStr = ' '.join(word_comment)

In [ ]:
#lemmatize to get similiar words together versus stemming which can just cut words shorter Ex: 'play', 'plays', 'played', and 'playing' have 'play' as the lemma. 
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

wordLemmatized = lemmatizer.lemmatize(wordStr)

In [ ]:
type(wordLemmatized)

str

In [ ]:
#put back into array so can be iterized for word frequency

lemmatized_array = wordLemmatized.split()


staying mei lin ross butterfly inn delightful house lovely natural light skylights wonderfully warm welcoming place mei lin especially kind loving fouryearold particularly loved helping feed bunnies g


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
type(wordLemmatized)

str

In [ ]:
#get frequency
word_freq = fd(lemmatized_array)

In [ ]:
word_freq

FreqDist({'place': 98569, 'great': 97784, 'stay': 95070, 'clean': 61479, 'nice': 43767, 'location': 43171, 'host': 40182, 'would': 39714, 'house': 36411, 'comfortable': 35997, ...})

In [ ]:
countOne = 0
for word in word_freq:
  if len(word) == 1:
    countOne += 1
print(countOne)

23


In [ ]:
word_freq.most_common(30)

[('place', 98569),
 ('great', 97784),
 ('stay', 95070),
 ('clean', 61479),
 ('nice', 43767),
 ('location', 43171),
 ('host', 40182),
 ('would', 39714),
 ('house', 36411),
 ('comfortable', 35997),
 ('room', 30002),
 ('home', 29825),
 ('everything', 27759),
 ('recommend', 26835),
 ('quiet', 25949),
 ('definitely', 24199),
 ('good', 22187),
 ('perfect', 21737),
 ('easy', 21647),
 ('area', 21095),
 ('well', 21069),
 ('really', 19639),
 ('space', 19375),
 ('time', 18349),
 ('beautiful', 18198),
 ('us', 17486),
 ('super', 17394),
 ('close', 16572),
 ('neighborhood', 16458),
 ('also', 15371)]

In [ ]:
# https://www.datacamp.com/tutorial/text-analytics-beginners-nltk
# https://realpython.com/python-keras-text-classification/
# https://stackabuse.com/text-classification-with-python-and-scikit-learn/
import numpy as np
import re

**Note, delete( or save seperately for own notes) above text preprocessing  since this would work as a word cloud in python but not needed to process outside of what we will do below for text classification**

In [100]:
#join reviews and listings together to get apartment (target variable) and comments ( our data to extract keywords from)
merged_df = listings_df.merge(reviews_df, on = 'listing_id', how = 'right')

In [101]:
merged_df.head()

,listing_id,name,neighbourhood,latitude,longitude,room_type,calculated_host_listings_count,license,review_id,date,reviewer_name,comments
0,4952.0,Butterfly Inn near Stanford with weekly cleaning,Palo Alto,37.43932,-122.15741,Private room,5.0,NaN,6560,2009-08-02,Diane,staying mei lin ross butterfly inn delightful ...
1,1353178.0,HackerHome - San Jose Dwtn: bunk 2,San Jose,37.34633,-121.88391,Shared room,37.0,NaN,7471795,2013-09-21,Jesus,definitely stay hackerhome go back san francis...
2,1353178.0,HackerHome - San Jose Dwtn: bunk 2,San Jose,37.34633,-121.88391,Shared room,37.0,NaN,10120145,2014-02-01,Jeanne,rob vic familyclass acts
3,1353178.0,HackerHome - San Jose Dwtn: bunk 2,San Jose,37.34633,-121.88391,Shared room,37.0,NaN,11096627,2014-03-20,Debora,nice place nice people nice location felt real...
4,1353178.0,HackerHome - San Jose Dwtn: bunk 2,San Jose,37.34633,-121.88391,Shared room,37.0,NaN,11875945,2014-04-17,Pascal,really good time hackerhome everyone really ea...


In [102]:
type(merged_df)

pandas.core.frame.DataFrame

In [105]:
merged_df.room_type.isnull().values.any()

True

In [106]:
# define variables containing the data and  target variable
x, y = merged_df.comments, merged_df.room_type

In [107]:
type(x)

pandas.core.series.Series

In [108]:
x.hasnans

False

In [109]:
#need to split and lemmatize
from nltk.stem import WordNetLemmatizer
import re
import numpy as np
lemmatizer = WordNetLemmatizer()
documents = []


#clean and lemmatize to create documents
for sen in range(0, len(x)):
    # Remove all the special characters
    document = re.sub(r'\W', ' ', str(x[sen]))
    
    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    
    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    
    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)
    
    # Converting to Lowercase
    document = document.lower()
    
    # Lemmatization
    document = document.split()

    document = [lemmatizer.lemmatize(word) for word in document]
    document = ' '.join(document)
    
    documents.append(document)

In [110]:
type(documents)

list

In [111]:
None in documents

False

In [112]:
#convert text to numbers for ML algorithms using Bag of Words
from sklearn.feature_extraction.text import CountVectorizer
#each unique word is converted to a feature so when limiting to 1500, that helps get rid of those with a low frequency
# min  df is minimum number of documents containing the feature
vectorizer = CountVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
#apply to x and transform the documents into numeric features using fit transform
x = vectorizer.fit_transform(documents).toarray()

In [113]:
type(x)

numpy.ndarray

In [114]:
findNulls(x)

294925500

In [47]:
#create tfidf from bag of words aka inverse document frequency = log( total # of ducoments/ # of documents containing word)
from sklearn.feature_extraction.text import TfidfTransformer
tdidfconverter = TfidfTransformer()

#now convert
x = tdidfconverter.fit_transform(x).toarray()

In [48]:
#split into training and testing
from sklearn.model_selection import train_test_split
#random state sets the seed and test size is a percentage so this is 20 80 split
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.2, random_state = 5)

In [54]:
type(x_train)

numpy.ndarray

In [53]:
#find if there are nulls 
def findNulls(data):
  return np.sum(~np.isnan(data))

In [55]:
findNulls(x_train)

235939500

In [58]:
x_train.shape

(157293, 1500)

In [59]:
print(x_train)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [60]:
# first 5 rows
print(x_train[0:5, :])

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


**Random Forest Classifier**

In [ ]:
# https://corporatefinanceinstitute.com/resources/data-science/random-forest/#:~:text=Among%20all%20the%20available%20classification,other%20classes%20in%20the%20data.
# https://stackabuse.com/text-classification-with-python-and-scikit-learn/
from sklearn.ensemble import RandomForestClassifier
rfclassifier = RandomForestClassifier(n_estimators = 1000, random_state = 5)
rfclassifier.fit(x_train,y_train)

ValueError: ignored

**SVM**

In [ ]:
from sklearn.svm import SVC
clf = SVC(gamma='auto')
clf.fit(x_train, y_train)

ValueError: ignored

CNN Convolutional Neural Network

In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(5, 2), random_state=1)
clf.fit(x, y)
MLPClassifier(alpha=1e-05, hidden_layer_sizes=(5, 2), random_state=1,
              solver='lbfgs')

ValueError: ignored